# Tinyflux: Example with fictious temperature measurements

## Import Python Libraries und Settings ⚙️

In [2]:
# Libraries
import pandas as pd
import csv
from tinyflux import TinyFlux, Point, FieldQuery, TagQuery, TimeQuery
from datetime import datetime, timezone, timedelta

# Settings
import warnings
warnings.filterwarnings("ignore")

# Abkürzungen für Query-Typen Globale Variablen 
time = TimeQuery()
tags = TagQuery()
field = FieldQuery()

## Load function for creating a dataframe from query results into memory

In [3]:
# Define the function to transform the list of Point objects into a DataFrame
def points_to_dataframe(points_list):
    data = []
    
    # Extract the relevant information
    for point in points_list:
        data.append({
            "time": point.time,  # Access time attribute
            "sensor_id": point.tags['sensor_id'],  # Access sender_id from tags dictionary
            "temperature": point.fields['temperature'],  # Access receiver_id from tags dictionary
            "status": point.tags['status']  # Access status from tags dictionary
        })

    # Create a DataFrame
    df = pd.DataFrame(data)

    # Convert time to datetime
    df['time'] = pd.to_datetime(df['time'])

    return df

## Initialize the TinyFlux database

In [4]:
# Erstellung der Tinyflux-Datenbank (wird bereitgestellt)
db = TinyFlux("temperatures.db")

## Importing a CSV file and show first descriptive statistics

In [5]:
csv_file = 'synthetic_temperature_readings.csv'
path_csv_file = 'example_data/' + csv_file


In [6]:
# CSV-Datei in Dataframe laden
df = pd.read_csv(path_csv_file)

# Das Format des Zeitstempels in Pandas setzen
df['time']= pd.to_datetime(df['time'], format='mixed')

df.head(10)



,time,sensor_id,temperature,status
0,2023-01-01 00:00:00,52,20.377761,operational
1,2023-01-01 08:44:41,93,2.217658,maintenance
2,2023-01-01 17:29:22,15,14.912385,offline
3,2023-01-02 02:14:03,72,6.517424,operational
4,2023-01-02 10:58:44,61,36.684591,operational
5,2023-01-02 19:43:25,21,-9.623282,operational
6,2023-01-03 04:28:06,83,1.266640,operational
7,2023-01-03 13:12:47,87,8.267841,operational
8,2023-01-03 21:57:28,75,14.390490,operational
9,2023-01-04 06:42:09,75,32.540876,operational


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   time         1000 non-null   datetime64[ns]
 1   sensor_id    1000 non-null   int64         
 2   temperature  1000 non-null   float64       
 3   status       1000 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 31.4+ KB


In [8]:
# Deskriptive Statistiken des Dataframes
df.describe(include='all')

# Eine Sensor Id ist keine Kategorie -> Auswertung nicht nötig

,time,sensor_id,temperature,status
count,1000,1000.000000,1000.000000,1000
unique,NaN,NaN,NaN,3
top,NaN,NaN,NaN,operational
freq,NaN,NaN,NaN,858
mean,2023-07-01 23:59:59.513999872,49.560000,15.392825,NaN
min,2023-01-01 00:00:00,1.000000,-9.988124,NaN
25%,2023-04-01 23:59:59.750000128,24.000000,2.945454,NaN
50%,2023-07-01 23:59:59.500000,50.000000,16.125006,NaN
75%,2023-09-30 23:59:59.249999872,75.000000,27.474836,NaN
max,2023-12-31 00:00:00,99.000000,39.967675,NaN


## Writing a CSV file into Tinyflux

In [9]:
# Read CSV and insert data into TinyFlux
with open(path_csv_file, mode="r") as file:
    reader = csv.DictReader(file)
    for row in reader:
        point = Point(
            time=datetime.strptime(row["time"], "%Y-%m-%d %H:%M:%S"),
            measurement="blockchain_transactions",
            fields={
                "temperature": float(row["temperature"])
                },  # Only numeric fields
            tags={
                "sensor_id": row["sensor_id"],
                "status": row["status"]  # Move non-numeric fields to tags
            }
        )
        db.insert(point)

## Writing manually into Tinyflux

In [10]:
# Länge der Datenbank
len(db)

13923

In [11]:
# Create manual records
p1 = Point(
    time=datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc),
    tags={"sensor_id": "20", "status": "operational"},
    fields={"temperature": 15.240971}
)

p2 = Point(
    time=datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc),
    tags={"sensor_id": "30", "status": "maintenance"},
    fields={"temperature": 20.517200}
)

# Insert into the DB.
db.insert_multiple([p1, p2])

2

In [12]:
# Länge der Datenbank
len(db)

13925

## Manuelles löschen von records

In [37]:
len(db)

13925

In [14]:
# manuelles löschen von records
# db.remove((tags.sensor_id == "20") & (time < datetime(2024, 1, 2, 0, 0, 0, tzinfo=timezone.utc)))


In [15]:
len(db)

13925

# Querys ----------------------

## Querying by Time

In [50]:
# Abfrage erster Eintrag und letzter Eintrag in der Datenbank
entries_before = db.search(time <= datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc))
entries_after = db.search(time >= datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc))

first_entry = entries_before[0] if entries_before else None
last_entry = entries_after[-1] if entries_after else None

print('Erster Eintrag:', first_entry)
print('Letzter Eintrag:', last_entry)


Erster Eintrag: Point(time=2023-01-01T00:00:00+00:00, measurement=blockchain_transactions, tags=sensor_id:52; status:operational, fields=temperature:20.37776111467864)
Letzter Eintrag: Point(time=2024-01-01T00:00:00+00:00, measurement=_default, tags=sensor_id:30; status:maintenance, fields=temperature:20.5172)


In [ ]:
# Definieren der Query (Daten aus dem Jahr 2023)

# Abfrage ab dem 01.01.2024
# time_query_conditions = (time >= datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc)) 

# Abfrage bis zum 01.01.2024
#time_query_conditions = (time <= datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc)) 

# Abfrage zwischen dem Date1 und dem Date2
time_query_conditions = (time >= datetime(2023, 1, 1, 0, 0, 0, tzinfo=timezone.utc)) & (time <= datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc))

# Datenabfrage mittels der definierten Query
my_results = db.search(time_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time = points_to_dataframe(my_results)
df_result_time

,time,sensor_id,temperature,status
0,2023-01-01 00:00:00+00:00,52,20.377761,operational
1,2023-01-01 00:00:00+00:00,52,20.377761,operational
2,2023-01-01 00:00:00+00:00,52,20.377761,operational
3,2023-01-01 00:00:00+00:00,52,20.377761,operational
4,2023-01-01 00:00:00+00:00,52,20.377761,operational
...,...,...,...,...
13920,2024-01-01 00:00:00+00:00,30,20.517200,maintenance
13921,2024-01-01 00:00:00+00:00,20,15.240971,operational
13922,2024-01-01 00:00:00+00:00,30,20.517200,maintenance
13923,2024-01-01 00:00:00+00:00,20,15.240971,operational


In [65]:
#anzahl der Einträge
len(df_result_time)

13925

In [76]:
df_result_time.describe(include='all')

,time,sensor_id,temperature,status
count,13925,13925,13925.000000,13925
unique,NaN,99,NaN,3
top,NaN,90,NaN,operational
freq,NaN,280,NaN,11942
mean,2023-07-02 06:41:08.016947968+00:00,NaN,15.350048,NaN
min,2023-01-01 00:00:00+00:00,NaN,-9.988124,NaN
25%,2023-04-01 08:41:48+00:00,NaN,2.852363,NaN
50%,2023-07-02 13:07:01+00:00,NaN,16.124052,NaN
75%,2023-10-01 15:18:11+00:00,NaN,27.393580,NaN
max,2024-01-01 00:00:00+00:00,NaN,39.967675,NaN


## Querying by Tag

In [91]:
# Abfrage aller verfügbaren keys
df.keys()

Index(['time', 'sensor_id', 'temperature', 'status'], dtype='object')

In [19]:
df.status.unique()

array(['operational', 'maintenance', 'offline'], dtype=object)

In [92]:
# Definieren der Queries
tag_query_conditions_operational = (tags.status == "operational")
tag_query_conditions_maintenance = (tags.status == "maintenance")
tag_query_conditions_offline = (tags.status == "offline")

# Datenabfrage mittels der definierten Queries
my_results_operational = db.search(tag_query_conditions_operational)
my_results_maintenance = db.search(tag_query_conditions_maintenance)
my_results_offline = db.search(tag_query_conditions_offline)

# Erstellung eines Dataframes aus den Query-Resultaten
df_result_tag_operational = points_to_dataframe(my_results_operational)
df_result_tag_maintenance = points_to_dataframe(my_results_maintenance)
df_result_tag_offline = points_to_dataframe(my_results_offline)

### Dataframe with operational status

In [21]:
df_result_tag_operational.head(2)

,time,sensor_id,temperature,status
0,2023-01-01 00:00:00+00:00,52,20.377761,operational
1,2023-01-01 00:00:00+00:00,52,20.377761,operational


In [22]:
df_result_tag_operational.describe(include='all')

,time,sensor_id,temperature,status
count,11942,11942,11942.000000,11942
unique,NaN,99,NaN,1
top,NaN,92,NaN,operational
freq,NaN,252,NaN,11942
mean,2023-07-01 00:08:45.098978304+00:00,NaN,15.581576,NaN
min,2023-01-01 00:00:00+00:00,NaN,-9.988124,NaN
25%,2023-03-31 23:57:07+00:00,NaN,3.557372,NaN
50%,2023-07-01 02:08:17+00:00,NaN,16.513257,NaN
75%,2023-09-30 04:19:27+00:00,NaN,27.621388,NaN
max,2024-01-01 00:00:00+00:00,NaN,39.967675,NaN


### Dataframe with maintenance status

In [23]:
df_result_tag_maintenance.head(2)

,time,sensor_id,temperature,status
0,2023-01-01 08:44:41+00:00,93,2.217658,maintenance
1,2023-01-01 08:44:41+00:00,93,2.217658,maintenance


In [24]:
df_result_tag_maintenance.describe(include='all')

,time,sensor_id,temperature,status
count,1241,1241,1241.000000,1241
unique,NaN,64,NaN,1
top,NaN,48,NaN,maintenance
freq,NaN,42,NaN,1241
mean,2023-07-09 08:16:00.465753344+00:00,NaN,13.476189,NaN
min,2023-01-01 08:44:41+00:00,NaN,-9.745893,NaN
25%,2023-04-22 11:53:30+00:00,NaN,2.217658,NaN
50%,2023-07-08 00:17:17+00:00,NaN,12.850011,NaN
75%,2023-10-11 11:24:41+00:00,NaN,26.892788,NaN
max,2024-01-01 00:00:00+00:00,NaN,39.606666,NaN


### Dataframe with offline status

In [25]:
df_result_tag_offline.head(2)

,time,sensor_id,temperature,status
0,2023-01-01 17:29:22+00:00,15,14.912385,offline
1,2023-01-01 17:29:22+00:00,15,14.912385,offline


In [26]:
df_result_tag_maintenance.describe(include='all')

,time,sensor_id,temperature,status
count,1241,1241,1241.000000,1241
unique,NaN,64,NaN,1
top,NaN,48,NaN,maintenance
freq,NaN,42,NaN,1241
mean,2023-07-09 08:16:00.465753344+00:00,NaN,13.476189,NaN
min,2023-01-01 08:44:41+00:00,NaN,-9.745893,NaN
25%,2023-04-22 11:53:30+00:00,NaN,2.217658,NaN
50%,2023-07-08 00:17:17+00:00,NaN,12.850011,NaN
75%,2023-10-11 11:24:41+00:00,NaN,26.892788,NaN
max,2024-01-01 00:00:00+00:00,NaN,39.606666,NaN


## Querying by Field

In [27]:
# Definieren der Queries
field_query_conditions = (field.temperature > 0)

# Datenabfrage mittels der definierten Queries
my_results = db.search(field_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_field = points_to_dataframe(my_results)

print("\n Dataframe with temperature measurements > 0:")
df_result_field.head(2)


 Dataframe with temperature measurements > 0:


,time,sensor_id,temperature,status
0,2023-01-01 00:00:00+00:00,52,20.377761,operational
1,2023-01-01 00:00:00+00:00,52,20.377761,operational


In [28]:
df_result_field.describe(include='all')

,time,sensor_id,temperature,status
count,11237,11237,11237.000000,11237
unique,NaN,99,NaN,3
top,NaN,62,NaN,operational
freq,NaN,252,NaN,9660
mean,2023-07-01 12:05:08.257363968+00:00,NaN,20.243782,NaN
min,2023-01-01 00:00:00+00:00,NaN,0.017957,NaN
25%,2023-03-29 10:44:19+00:00,NaN,11.088133,NaN
50%,2023-07-01 10:52:58+00:00,NaN,19.914303,NaN
75%,2023-09-30 04:19:27+00:00,NaN,29.687193,NaN
max,2024-01-01 00:00:00+00:00,NaN,39.967675,NaN


# Advanced Querys -------------------------

Variations of combining queries

### Querying by Time and Tag

In [106]:
# Zeige alle Keys 
df.keys()

Index(['time', 'sensor_id', 'temperature', 'status'], dtype='object')

In [107]:
# Zeige die unique values eines Keys
key = 'status'
df[key].unique()

array(['operational', 'maintenance', 'offline'], dtype=object)

In [110]:
# Querying all temperatures, that were measured before Mach 31 2023 with the status "operational"

# Definieren der Queries

# Zeit vor dem 31. März 2023
# time_query_conditions = (time < datetime(2023, 3, 31, 0, 0, 0, tzinfo=timezone.utc))
#  
# Zeit nach dem 31. März 2023
time_query_conditions = (time > datetime(2023, 3, 31, 0, 0, 0, tzinfo=timezone.utc)) 

# Abfrage zwischen dem Date1 und dem Date2
# time_query_conditions = (time >= datetime(2023, 1, 1, 0, 0, 0, tzinfo=timezone.utc)) & (time <= datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc)) 

tag_query_conditions = (tags.status == "maintenance")

# Datenabfrage mittels der definierten Queries
my_results = db.search(time_query_conditions & tag_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time_tag = points_to_dataframe(my_results)
df_result_time_tag.head(2)

,time,sensor_id,temperature,status
0,2023-04-14 11:30:27+00:00,16,14.873245,maintenance
1,2023-04-14 11:30:27+00:00,16,14.873245,maintenance


In [105]:
df_result_time_tag.describe(include='all')

,time,sensor_id,temperature,status
count,961,961,961.000000,961
unique,NaN,54,NaN,1
top,NaN,99,NaN,maintenance
freq,NaN,42,NaN,961
mean,2023-08-20 16:07:50.553590016+00:00,NaN,13.686556,NaN
min,2023-04-14 11:30:27+00:00,NaN,-9.745893,NaN
25%,2023-06-20 21:17:07+00:00,NaN,2.376998,NaN
50%,2023-08-14 04:15:08+00:00,NaN,13.426971,NaN
75%,2023-10-28 14:24:51+00:00,NaN,26.892788,NaN
max,2024-01-01 00:00:00+00:00,NaN,39.606666,NaN


### Querying by Tag and Field

In [ ]:
df.keys() # field / keys

Index(['time', 'sensor_id', 'temperature', 'status'], dtype='object')

In [116]:
df.status.unique()

array(['operational', 'maintenance', 'offline'], dtype=object)

In [130]:
# Querying all temperatures higher than 20 degrees and with the status "maintenance"

# Definieren der Queries
tag_query_conditions = (tags.status == "maintenance")
# field_query_conditions = (field.temperature > 20.0) # grösser als 20
field_query_conditions = (field.temperature < 20.0) # kleiner als 20

# Datenabfrage mittels der definierten Queries
my_results = db.search(tag_query_conditions & field_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_tag_field = points_to_dataframe(my_results)
df_result_tag_field.head(5)

,time,sensor_id,temperature,status
0,2023-01-01 08:44:41+00:00,93,2.217658,maintenance
1,2023-01-01 08:44:41+00:00,93,2.217658,maintenance
2,2023-01-01 08:44:41+00:00,93,2.217658,maintenance
3,2023-01-01 08:44:41+00:00,93,2.217658,maintenance
4,2023-01-01 08:44:41+00:00,93,2.217658,maintenance


In [32]:
df_result_tag_field.describe(include='all')

,time,sensor_id,temperature,status
count,441,441,441.000000,441
unique,NaN,29,NaN,1
top,NaN,62,NaN,maintenance
freq,NaN,28,NaN,441
mean,2023-07-21 15:16:42.158730240+00:00,NaN,29.916453,NaN
min,2023-01-09 00:23:03+00:00,NaN,20.517200,NaN
25%,2023-04-23 14:07:34+00:00,NaN,26.695496,NaN
50%,2023-08-07 14:50:48+00:00,NaN,29.989055,NaN
75%,2023-10-18 09:33:41+00:00,NaN,34.081857,NaN
max,2024-01-01 00:00:00+00:00,NaN,39.606666,NaN


## Variations of queries with multiple AND and OR conditions

### ⛄ Querying all measurements during winter months with temperatures below 0

Definieren der Queries 

or |     
und &

In [131]:
# Definieren der Queries
time_query_conditions = (time <= datetime(2023, 2, 28, 0, 0, 0, tzinfo=timezone.utc)) | (time >= datetime(2023, 12, 1, tzinfo=timezone.utc)) # Abfrage der Wintermonate
field_query_conditions = (field.temperature < 0.0) # kleiner als 0

# Datenabfrage mittels der definierten Queries
my_results = db.search(time_query_conditions & field_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time_field = points_to_dataframe(my_results)
df_result_time_field.head(2)

,time,sensor_id,temperature,status
0,2023-01-02 19:43:25+00:00,21,-9.623282,operational
1,2023-01-02 19:43:25+00:00,21,-9.623282,operational


In [34]:
df_result_time_field.describe(include='all')

,time,sensor_id,temperature,status
count,644,644,644.000000,644
unique,NaN,32,NaN,3
top,NaN,90,NaN,operational
freq,NaN,56,NaN,546
mean,2023-05-22 15:02:38.826087168+00:00,NaN,-5.301463,NaN
min,2023-01-02 19:43:25+00:00,NaN,-9.938166,NaN
25%,2023-01-19 22:43:36+00:00,NaN,-7.383056,NaN
50%,2023-02-20 19:53:30.500000+00:00,NaN,-5.457587,NaN
75%,2023-12-12 18:45:45+00:00,NaN,-3.044190,NaN
max,2023-12-28 10:47:12+00:00,NaN,-0.024654,NaN


### ☀️ Querying all measurements during summer months with temperatures above 0

In [ ]:
# Definieren der Queries
time_query_conditions = (time <= datetime(2023, 8, 31, 0, 0, 0, tzinfo=timezone.utc)) & (time >= datetime(2023, 6, 1, tzinfo=timezone.utc))
field_query_conditions = (field.temperature > 0.0)

# Datenabfrage mittels der definierten Queries
my_results = db.search(time_query_conditions & field_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time_field_2 = points_to_dataframe(my_results)
df_result_time_field_2.head(2)

,time,sensor_id,temperature,status
0,2023-06-01 05:04:08+00:00,60,35.516955,operational
1,2023-06-01 05:04:08+00:00,60,35.516955,operational


In [36]:
df_result_time_field_2.describe(include='all')

,time,sensor_id,temperature,status
count,2732,2732,2732.000000,2732
unique,NaN,83,NaN,3
top,NaN,58,NaN,operational
freq,NaN,126,NaN,2326
mean,2023-07-17 12:45:09.387262208+00:00,NaN,21.817459,NaN
min,2023-06-01 05:04:08+00:00,NaN,0.017957,NaN
25%,2023-06-23 19:14:35+00:00,NaN,13.104865,NaN
50%,2023-07-18 13:53:09+00:00,NaN,22.514709,NaN
75%,2023-08-10 04:03:36+00:00,NaN,32.006075,NaN
max,2023-08-30 22:30:37+00:00,NaN,39.816789,NaN
